# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 9 2022 8:30AM,243716,10,0085925778,ISDIN Corporation,Released
1,Aug 9 2022 8:30AM,243716,10,0085928352,ISDIN Corporation,Released
2,Aug 9 2022 8:30AM,243716,10,0085928425,ISDIN Corporation,Released
3,Aug 9 2022 8:30AM,243716,10,0085928434,ISDIN Corporation,Released
4,Aug 9 2022 8:30AM,243716,10,0085928451,ISDIN Corporation,Released
5,Aug 9 2022 8:30AM,243716,10,0085928459,ISDIN Corporation,Released
6,Aug 9 2022 8:30AM,243716,10,0085928467,ISDIN Corporation,Released
7,Aug 9 2022 8:30AM,243716,10,0085928471,ISDIN Corporation,Released
8,Aug 9 2022 8:30AM,243716,10,0085928480,ISDIN Corporation,Released
9,Aug 9 2022 8:30AM,243716,10,0085928498,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,243726,Released,4
26,243727,Released,1
27,243728,Released,15
28,243729,Released,1
29,243730,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243726,NaN,4.0
243727,NaN,1.0
243728,NaN,15.0
243729,NaN,1.0
243730,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243677,0.0,1.0
243682,0.0,1.0
243684,13.0,4.0
243687,14.0,1.0
243689,6.0,28.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243677,0,1
243682,0,1
243684,13,4
243687,14,1
243689,6,28


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243677,0,1
1,243682,0,1
2,243684,13,4
3,243687,14,1
4,243689,6,28


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243677,,1
1,243682,,1
2,243684,13,4
3,243687,14,1
4,243689,6,28


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation
15,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation
33,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation
39,Aug 9 2022 8:24AM,243730,10,Emerginnova
40,Aug 9 2022 8:19AM,243728,10,ISDIN Corporation
55,Aug 9 2022 8:18AM,243729,16,"Emersa Waterbox, LLC"
56,Aug 9 2022 7:43AM,243727,19,"AdvaGen Pharma, Ltd"
57,Aug 9 2022 7:39AM,243726,19,"AdvaGen Pharma, Ltd"
61,Aug 8 2022 5:49PM,243722,21,"NBTY Global, Inc."
62,Aug 8 2022 5:49PM,243721,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,,15
1,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,,18
2,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,,6
3,Aug 9 2022 8:24AM,243730,10,Emerginnova,,1
4,Aug 9 2022 8:19AM,243728,10,ISDIN Corporation,,15
5,Aug 9 2022 8:18AM,243729,16,"Emersa Waterbox, LLC",,1
6,Aug 9 2022 7:43AM,243727,19,"AdvaGen Pharma, Ltd",,1
7,Aug 9 2022 7:39AM,243726,19,"AdvaGen Pharma, Ltd",,4
8,Aug 8 2022 5:49PM,243722,21,"NBTY Global, Inc.",,1
9,Aug 8 2022 5:49PM,243721,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,15,
1,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,18,
2,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,6,
3,Aug 9 2022 8:24AM,243730,10,Emerginnova,1,
4,Aug 9 2022 8:19AM,243728,10,ISDIN Corporation,15,
5,Aug 9 2022 8:18AM,243729,16,"Emersa Waterbox, LLC",1,
6,Aug 9 2022 7:43AM,243727,19,"AdvaGen Pharma, Ltd",1,
7,Aug 9 2022 7:39AM,243726,19,"AdvaGen Pharma, Ltd",4,
8,Aug 8 2022 5:49PM,243722,21,"NBTY Global, Inc.",1,
9,Aug 8 2022 5:49PM,243721,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,15,
1,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,18,
2,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,6,
3,Aug 9 2022 8:24AM,243730,10,Emerginnova,1,
4,Aug 9 2022 8:19AM,243728,10,ISDIN Corporation,15,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,15.0,NaN
1,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,18.0,NaN
2,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,6.0,NaN
3,Aug 9 2022 8:24AM,243730,10,Emerginnova,1.0,NaN
4,Aug 9 2022 8:19AM,243728,10,ISDIN Corporation,15.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,15.0,0.0
1,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,18.0,0.0
2,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,6.0,0.0
3,Aug 9 2022 8:24AM,243730,10,Emerginnova,1.0,0.0
4,Aug 9 2022 8:19AM,243728,10,ISDIN Corporation,15.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2193372,59.0,0.0
102,974861,4.0,0.0
16,487432,1.0,1.0
19,974859,7.0,0.0
20,1218467,49.0,33.0
21,731163,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2193372,59.0,0.0
1,102,974861,4.0,0.0
2,16,487432,1.0,1.0
3,19,974859,7.0,0.0
4,20,1218467,49.0,33.0
5,21,731163,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,59.0,0.0
1,102,4.0,0.0
2,16,1.0,1.0
3,19,7.0,0.0
4,20,49.0,33.0
5,21,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,59.0
1,102,Released,4.0
2,16,Released,1.0
3,19,Released,7.0
4,20,Released,49.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,16,19,20,21
Status,,,,,,
Executing,0.0,0.0,1.0,0.0,33.0,0.0
Released,59.0,4.0,1.0,7.0,49.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,16,19,20,21
0,Executing,0.0,0.0,1.0,0.0,33.0,0.0
1,Released,59.0,4.0,1.0,7.0,49.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,16,19,20,21
0,Executing,0.0,0.0,1.0,0.0,33.0,0.0
1,Released,59.0,4.0,1.0,7.0,49.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()